In [147]:
# Import our libraries
import pandas as pd
import numpy as np
# Import sklearn libraries
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold, cross_val_score
from sklearn.model_selection import cross_validate
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_curve, precision_recall_curve, auc, make_scorer, confusion_matrix, f1_score, fbeta_score
# Import the Naive Bayes, logistic regression, Bagging, RandomForest, AdaBoost, GradientBoost, Decision Trees and SVM Classifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn import svm
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
#from matplotlib import style
#plt.style.use('bmh')
#plt.style.use('ggplot')
plt.style.use('seaborn-notebook')
from matplotlib.ticker import StrMethodFormatter
from sklearn.preprocessing import StandardScaler, MinMaxScaler, LabelBinarizer

In [148]:
train = pd.read_csv("development_dataset.csv")
test1 = pd.read_csv("leaderboard_dataset.csv")
test2 = pd.read_csv("Evaluation_dataset.csv")

In [149]:
train["VAR14"] = train["VAR14"].replace('.', '13')
train["VAR14"] = train["VAR14"].astype(float)
test1["VAR14"] = test1["VAR14"].replace('.', '13')
test1["VAR14"] = test1["VAR14"].astype(float)
test2["VAR14"] = test2["VAR14"].replace('.', '13')
test2["VAR14"] = test2["VAR14"].astype(float)

In [150]:
data = [train,test1]
columns = ["VAR2","VAR3","VAR4","VAR6","VAR7","VAR8","VAR9","VAR10","VAR11","VAR12","VAR13","VAR15","VAR16","VAR17"]
for dataset in data:
    for x in columns:
        mean = dataset[x].mean()
        std = dataset[x].std()
        is_null = dataset[x].isnull().sum()
        # compute random numbers between the mean, std and is_null
        rand_var2 = np.random.randint(mean - std, mean + std, size = is_null)
        # fill NaN values in Age column with random values generated
        age_slice = dataset[x].copy()
        age_slice[np.isnan(age_slice)] = rand_var2
        dataset[x] = age_slice
        dataset[x] = dataset[x].astype(float)
dataset[x].isnull().sum()

0

In [151]:
data = [train]
for dataset in data:
    mean = train["VAR5"].mean()
    std = train["VAR5"].std()
    is_null = dataset["VAR5"].isnull().sum()
    # compute random numbers between the mean, std and is_null
    rand_var2 = np.random.randint(mean - std, mean + std, size = is_null)
    # fill NaN values in Age column with random values generated
    age_slice = dataset["VAR5"].copy()
    age_slice[np.isnan(age_slice)] = rand_var2
    dataset["VAR5"] = age_slice
    dataset["VAR5"] = train["VAR5"].astype(float)
train["VAR5"].isnull().sum()

0

In [152]:
credit_line = {"Low": 1, "Medium": 2, "High": 3}
train["VAR21"] = train["VAR21"].map(credit_line)
train.head()

,VAR1,VAR2,VAR3,VAR4,VAR5,VAR6,VAR7,VAR8,VAR9,VAR10,...,VAR12,VAR13,VAR14,VAR15,VAR16,VAR17,VAR18,VAR19,VAR20,VAR21
0,1,828.235294,0.138889,321.428571,625.911006,1.818182,1.081550,198.113469,49.000000,58.632548,...,100.083403,1.0,1.0,1.0,1.540594,1077.0,1,0,100.104991,1
1,2,911.764706,0.027778,392.000000,611.574748,8.181818,1.344479,198.600020,22.086661,53.000000,...,15.012510,19.0,1.0,3.0,1.614613,906.0,0,1,146.654045,3
2,3,962.352941,0.833333,35.714286,615.825381,8.181818,0.720796,197.267767,-19.000000,58.632548,...,210.175146,10.0,1.0,10.0,1.044599,857.0,0,0,98.249570,2
3,4,892.941177,-3.000000,177.000000,638.076431,9.090909,0.820218,197.355744,4.363431,58.632548,...,127.000000,4.0,1.0,7.0,1.145729,718.0,1,0,140.862306,1
4,5,914.117647,0.083333,134.000000,626.514988,5.181818,1.372928,198.790477,85.938202,58.632548,...,210.175146,15.0,13.0,7.0,1.558341,921.0,1,0,101.268503,3


In [153]:
train["VAR21"] = train["VAR21"].astype(float)

In [154]:
train_numerical_features = list(train.select_dtypes(include=['int64', 'float64', 'int32']).columns)
ss_scaler = StandardScaler()
train_ss = pd.DataFrame(data = train)
train_ss[train_numerical_features] = ss_scaler.fit_transform(train_ss[train_numerical_features])

C:\Users\Shridam\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:625: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\Users\Shridam\Anaconda3\lib\site-packages\sklearn\base.py:462: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)


In [155]:
train_ss.head()

,VAR1,VAR2,VAR3,VAR4,VAR5,VAR6,VAR7,VAR8,VAR9,VAR10,...,VAR12,VAR13,VAR14,VAR15,VAR16,VAR17,VAR18,VAR19,VAR20,VAR21
0,-1.732000,-1.791514,-0.075932,0.354385,-0.183402,-0.478570,-0.109163,-0.166796,-0.070249,0.345246,...,-0.158700,-1.027186,-0.411577,-1.011465,-0.068682,1.297193,0.876074,-0.651629,-0.497049,-1.185529
1,-1.731898,-0.112725,-0.084992,0.690959,-0.647924,-0.252013,-0.079560,-0.123318,-0.453717,-0.513220,...,-0.949969,0.754114,-0.411577,-0.831058,-0.059405,0.430701,-0.584121,1.534615,-0.119305,1.955435
2,-1.731796,0.904006,-0.019306,-1.008261,-0.510195,-0.252013,-0.149780,-0.242367,-1.039131,0.345246,...,0.865295,-0.136536,-0.411577,-0.199633,-0.130852,0.182408,-0.584121,-0.651629,-0.512105,0.384953
3,-1.731694,-0.491044,-0.331880,-0.334432,0.210781,-0.219647,-0.138587,-0.234506,-0.706242,0.345246,...,0.091659,-0.730303,-0.411577,-0.470244,-0.118176,-0.521934,0.876074,-0.651629,-0.166305,-1.185529
4,-1.731592,-0.065436,-0.080462,-0.539511,-0.163832,-0.358818,-0.076357,-0.106299,0.456056,0.345246,...,0.865295,0.358270,7.447065,-0.470244,-0.066458,0.506709,0.876074,-0.651629,-0.487607,1.955435


In [156]:
encode_col_list = list(train.select_dtypes(include=['object']).columns)
for i in encode_col_list:
    train_ss = pd.concat([train_ss,pd.get_dummies(train_ss[i], prefix=i)],axis=1)
    train_ss.drop(i, axis = 1, inplace=True)

In [157]:
train_ss = train_ss.drop(["VAR1"], axis=1)
X_train = train_ss.drop("VAR21", axis=1)
Y_train = train_ss["VAR21"]
X_test1  = test1.drop("VAR1", axis=1).copy()
from sklearn import preprocessing
from sklearn import utils

lab_enc = preprocessing.LabelEncoder()
encoded = lab_enc.fit_transform(Y_train)
logreg = LogisticRegression()
logreg.fit(X_train, encoded)

Y_pred7 = logreg.predict(X_test1)

acc_log = round(logreg.score(X_train, encoded) * 100, 2)
print (acc_log)

C:\Users\Shridam\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\Shridam\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


55.19


In [158]:
encoded


array([0, 2, 1, ..., 0, 0, 2], dtype=int64)

In [159]:
predictions7 = Y_pred7
submission = pd.DataFrame({'no.':test1['VAR1'], 'cl':predictions7})
credit_line_reverse = {1.0: "Low", 2.0 : "Medium", 3.0 : "High"}
submission["cl"] = submission["cl"].map(credit_line_reverse)
filename = 'Ezio_IITGuwahati_8.csv'
submission.to_csv(filename, index = False, header = None)
print('saved file: ' + filename)


saved file: Ezio_IITGuwahati_8.csv
